# Project 5
# Car price prediction model

## Подготовка

Загружаем необходимые библиотеки, настраиваем отображение, инициализируем глобальные переменные

In [1]:
# imports
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
# options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# global definitions
RANDOM_STATE = 73

Загружаем подготовленный тестовый дата-сет

In [2]:
df_test = pd.read_csv('test_clean.csv')
df_test.head()

,body_type,brand,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,mileage,model_date,model_name,doors_num,parsing_unixtime,production_date,transmission,vendor,num_owners,ownership,docs,drive,steering
0,LIFTBACK,SKODA,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2,105,"{""engine-proof"":true,""tinted-glass"":true,""airb...",GASOLINE,74000,2013,OCTAVIA,5,1603226273,2014,ROBOT,0,3,NaN,0,FORWARD_CONTROL,0
1,LIFTBACK,SKODA,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",GASOLINE,60563,2017,OCTAVIA,5,1603226277,2017,MECHANICAL,0,1,NaN,0,FORWARD_CONTROL,0
2,LIFTBACK,SKODA,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8,152,"{""cruise-control"":true,""tinted-glass"":true,""es...",GASOLINE,88000,2013,SUPERB,5,1603226280,2014,ROBOT,0,1,NaN,0,FORWARD_CONTROL,0
3,LIFTBACK,SKODA,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110,"{""cruise-control"":true,""roller-blind-for-rear-...",GASOLINE,95000,2013,OCTAVIA,5,1603226284,2014,AUTOMATIC,0,1,NaN,0,FORWARD_CONTROL,0
4,LIFTBACK,SKODA,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",GASOLINE,58536,2008,OCTAVIA,5,1603226288,2012,AUTOMATIC,0,1,NaN,0,FORWARD_CONTROL,0


## 2. Выгрузка данных с auto.ru

In [3]:
max_time = pd.to_datetime(df_test.parsing_unixtime.max(), unit='s')

Итак, машины каких возрастов необходимо искать?

In [4]:
df_test.production_date.max(), df_test.production_date.min()

(2020, 1904)

Опа, аж с 1904 года модели есть. пожалуй, нет смысла фильтровать запросы по годам

Теперь посмотрим, какие модели надо искать:

In [5]:
brands = list(df_test.brand.unique())
brands

['SKODA',
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

Брэндов не так уж и много, все они либо Германия (Чехия туда же) или Япония. Для полноты картины возьмем побольше брэндов, чтобы можно было понять, улучшится ли модель, если использовать только брэнды из тестового набора, или нет. 

In [6]:
brands_more = ['AUDI', 'BMW', 'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER',
              'CITROEN', 'DAEWOO', 'DODGE', 'FORD', 'GEELY', 'HONDA', 'HYUNDAI',
              'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS', 'MAZDA', 'MINI',
              'MITSUBISHI', 'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT',
              'SKODA', 'SUBARU', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO',
              'GREAT_WALL', 'LAND_ROVER', 'MERCEDES', 'SSANG_YONG']

Пдоготовим данные для запроса с сайта

In [7]:
# базовый адрес всех запросов
url = 'https://auto.ru/-/ajax/desktop/listing/'
# маскируем запросы под более-менее реальнй браузер
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9',
    'Connection': 'keep-alive',
    'Content-Length': '88',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=a15ced727ae7d4ef1accda3dbdee74d1e45931e283d946ed; autoru_sid=a%3Ag60f5792323iu07g47e3q40o7hufj5bl.d4ed40f4845ba04c8cce077207215b0e%7C1626700067837.604800.ayAhiHukwB5Kfnf2wlLXLA.7Kq3GdBNqOTdRMPOQW0P1JZQuQkbdsqNKksC15-30Ts; autoruuid=g60f5792323iu07g47e3q40o7hufj5bl.d4ed40f4845ba04c8cce077207215b0e; suid=856890532234352dca31bf8b0f2a6e35.daf95c11739e31e93195dcf6fe99c996; from=direct; X-Vertis-DC=sas; yuidlt=1; yandexuid=408671381626700067; gdpr=0; _ym_isad=2; _ym_uid=1626700073813446272; gids=213; gradius=300; cycada=P1s14aIVv8SCmtti44Bf9U2DYI9zsumdr7ZGMj7dIXM=; from_lifetime=1626700099526; _ym_d=1626700099',
    'DNT': '1',
    'Host': 'auto.ru',
    'Origin': 'https://auto.ru',
    'Referer': 'https://auto.ru/moskva/cars/all/',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'same-origin',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'x-client-app-version': '743f1b0211f',
    'x-client-date': '1626700102145',
    'x-csrf-token': 'a15ced727ae7d4ef1accda3dbdee74d1e45931e283d946ed',
    'x-page-request-id': '33f5813c3cb7333a18944cf014727646',
    'x-requested-with': 'fetch',
}
# запрос первой порции объявлений
req_data = {
    'section':'all',
    'category':'cars',
    'catalog_filter':[{'mark': brands[0]}],
    'output_type':'list',
    'geo_radius':300,
    'geo_id':[213]
}

Делаем первый запрос

In [9]:
response = requests.post(url, headers=headers, json=req_data)
obj0 = response.json()

И смотрим, что у нас содержится в ответе. Какие из этих данных нам нужны, и как их парсить и приводить к нужному формату.

Объявления представлены в массиве объектов под ключом `offers`. Для простоты визуального анализа я заранее удалил большой и не несущий особого смысла для нас объект под ключом `image_urls`, в котором записан масив ссылок на фотографии авто.

In [10]:
offer0 = obj0['offers'][0]
offer0['state']['image_urls'] = None
for key in offer0.keys():
    print('=======', key, '=======')
    pprint(offer0[key])

======= availability =======
'IN_STOCK'
======= category =======
'cars'
======= color_hex =======
'C49648'
======= delivery_info =======
{}
======= description =======
('В стоимость автомобиля включено дополнительное оборудование дилера на сумму '
 '154900 р., подробную информацию уточняйте в отделе продаж\n'
 '\n'
 '———————————————————————————\n'
 'ФУНКЦИОНАЛЬНОЕ ОБОРУДОВАНИЕ\n'
 '———————————————————————————\n'
 '• Центральный замок с дистанционным управлением\n'
 '• Галогеновые фары с LED-ходовыми огнями\n'
 '• Дневной свет фар Day Light\n'
 '• Наружные зеркала с обогревом\n'
 '• Передние сиденья с подогревом\n'
 '• Наружные зеркала с электрорегулировкой\n'
 '• Электропривод стекол: есть\n'
 '• Усилитель руля, с регулировкой в зависимости от скорости\n'
 '• Регулировка руля: есть\n'
 '• Многофункциональное рулевое колесо с управлением радио и телефоном\n'
 '• Система кондиционирования: кондиционер\n'
 '• Радио Swing - 2DIN, SD, USB-C, MP3\n'
 '• 8 динамиков\n'
 '• Складное заднее сид

Я опущу описание процесса выбора полей для выгрузки, в методе ниже все довольно прозрачно обозначено. Все нужные данные в ответном сообщении есть, ниже представлен метод, который парсит отдельно взятое объявление и добавляет новый объект-строку в массив строк будущего дата-сета

In [11]:
def parse_offer(arr, offer):
    try:
        docs = offer['documents']
        # пропускаем растаможенные
        if (not docs['custom_cleared']):
            return
        # пропускаем битые
        if (not offer['state']['state_not_beaten']):
            return
        # ищем только цены в рублях
        if (not 'RUR' in offer['price_info']):
            return
        # парсим остальные поля
        veh_info = offer['vehicle_info']
        tech_param = veh_info['tech_param']
        arr.append({
            'body_type': veh_info['configuration']['body_type'],
            'brand': veh_info['mark_info']['code'],
            'color': offer['color_hex'],
            'complectation': veh_info['complectation'],
            'description': (offer['description'] if 'description' in offer else None),
            'engine_vol': np.round(tech_param['displacement'] / 1000.0, 1),
            'engine_power': tech_param['power'],
            'equipment': veh_info['equipment'],
            'fuel_type': tech_param['engine_type'],
            'mileage': offer['state']['mileage'],
            'model_date': veh_info['super_gen']['year_from'],
            'model_name': veh_info['model_info']['code'],
            'doors_num': veh_info['configuration']['doors_count'],
            'production_date': docs['year'],
            'transmission': tech_param['transmission'],
            'vendor': int(veh_info['vendor'] == 'JAPANESE'),
            'num_owners': (docs['owners_number'] if 'owners_number' in docs else 0),
            'ownership': None,
            'docs': int((not docs['pts'] == 'ORIGINAL') if 'pts' in docs else False),
            'drive': tech_param['gear_type'],
            'steering': int(veh_info['steering_wheel'] == 'RIGHT'),
            'price': offer['price_info']['RUR'],
            'id': offer['id']
        })
    except: return # do nothing, supress error
    

def parse_offers(arr, obj):
    old_len = len(arr)
    offers = obj['offers']
    for offer in offers:
        parse_offer(arr, offer)
        
    return len(arr) - old_len

Создаем массив строк и парсим результат первого запроса. Проверяем, сколько же объявлений успешно распарсилось

In [12]:
arr = list()
parse_offers(arr, obj0)

37

Смотрим, как в API `auto.tu` устроен процесс дозапроса объявлений. За это отвечает объект под ключом `pagination`

In [13]:
obj0['pagination']

{'total_page_count': 99,
 'total_offers_count': 3699,
 'page': 1,
 'page_size': 37,
 'from': 1,
 'to': 37,
 'current': 1}

In [14]:
def parse_by_brand(arr, brand):
    print(f'Starting to collect offers by brand "{brand}"')
    global url
    global headers
    global req_data    
    # prepare local request data
    req_data_m = req_data.copy()
    req_data_m['catalog_filter'] = [{'mark': brand}]
    # make first request
    response = requests.post(url, headers=headers, json=req_data_m)
    obj = response.json()
    num_pages = obj['pagination']['total_page_count']
    # parse first page
    n = parse_offers(arr, obj)
    print(f'Page 1 of {num_pages}: parsed {n} offers -> {len(arr)} total', end='\r')
    time_sleep = 0
    page = 2
    error_cnt = 0
    while page <= num_pages: # потому что нумерация не с 0, а с 1
        time.sleep(1.23) # немного подождем
        try:
            # обновляем запрос
            req_data_m['page'] = page
            # получаем объявления
            response = requests.post(url, headers=headers, json=req_data_m)
            obj = response.json()
            # парсим
            n = parse_offers(arr, obj)
            print(f'Page {page} of {num_pages}: parsed {n} offers -> {len(arr)} total', end='\r')
            # обновляем переменные
            time_sleep = 0
            error_cnt = 0
            page += 1
        except Exception as ex:
            error_cnt += 1
            time_sleep += 5
            if (error_cnt < 3):
                print(f'Page {page} of {num_pages}: error [{type(ex)}], wait {time_sleep} seconds and retry')
            else:
                page += 1
                print(f'Page {page} of {num_pages}: error [{type(ex)}], wait {time_sleep} seconds and skip this page')
                
            print(ex)
            time.sleep(time_sleep)

In [15]:
for m in brands_more:
    parse_by_brand(arr, m)

Starting to collect offers by brand "AUDI"
Starting to collect offers by brand "BMW"otal
Starting to collect offers by brand "CADILLAC"
Starting to collect offers by brand "CHERY"al
Starting to collect offers by brand "CHEVROLET"
Starting to collect offers by brand "CHRYSLER"
Starting to collect offers by brand "CITROEN"
Starting to collect offers by brand "DAEWOO"al
Starting to collect offers by brand "DODGE"tal
Starting to collect offers by brand "FORD"al
Starting to collect offers by brand "GEELY"tal
Starting to collect offers by brand "HONDA"tal
Page 19 of 28: error [<class 'KeyError'>], wait 5 seconds and retry
'tech_param'
Page 19 of 28: error [<class 'KeyError'>], wait 10 seconds and retry
'tech_param'
Page 20 of 28: error [<class 'KeyError'>], wait 15 seconds and skip this page
'tech_param'
Starting to collect offers by brand "HYUNDAI"l
Starting to collect offers by brand "INFINITI"
Starting to collect offers by brand "JAGUAR"ll
Starting to collect offers by brand "JEEP"al
Star

In [16]:
len(arr)

67189

Итак, у нас получилось более 67 тысяч свежих объявлений. Создадим из них тренировочный дата-сет и посмотрим на него.

In [17]:
df_columns = list(df_test.columns)
df_columns.append('price')
df_columns.append('id')
df = pd.DataFrame(arr, columns=df_columns)

In [18]:
df.head()

,body_type,brand,color,complectation,description,engine_vol,engine_power,equipment,fuel_type,mileage,model_date,model_name,doors_num,parsing_unixtime,production_date,transmission,vendor,num_owners,ownership,docs,drive,steering,price,id
0,ALLROAD_5_DOORS,SKODA,C49648,"{'id': '21837046', 'name': 'Active (2020-2021)...",В стоимость автомобиля включено дополнительное...,1.4,150,"{'engine-proof': True, 'asr': True, 'glonass':...",GASOLINE,0,2017,KAROQ,5,NaN,2021,AUTOMATIC,0,0,None,0,FORWARD_CONTROL,0,1915400,1103478587
1,LIFTBACK,SKODA,200204,"{'id': '2436032', 'name': 'Elegance', 'availab...",Причина продажи — куплен новый автомобиль \nКо...,1.8,152,"{'cruise-control': True, 'asr': True, 'tinted-...",GASOLINE,179000,2008,SUPERB,5,NaN,2012,AUTOMATIC,0,2,None,0,FORWARD_CONTROL,0,880000,1104616093
2,LIFTBACK,SKODA,CACECB,{'id': '0'},Продам Суперба ! В отличном техническом состоя...,1.4,150,{},GASOLINE,83500,2015,SUPERB,5,NaN,2015,ROBOT,0,3,None,0,FORWARD_CONTROL,0,1200000,1102548494
3,LIFTBACK,SKODA,97948F,"{'id': '22128588', 'name': 'Ambition', 'availa...",ЛОТ: 01267611\n\nАвтомобили в наличии и с ПТС ...,1.6,110,"{'cruise-control': True, 'glonass': True, 'esp...",GASOLINE,0,2020,RAPID,5,NaN,2021,AUTOMATIC,0,0,None,0,FORWARD_CONTROL,0,1250800,1104705443
4,ALLROAD_5_DOORS,SKODA,FAFBFB,{'id': '0'},"Все автомобили, для Вашего удобства, находятся...",1.4,150,"{'cruise-control': True, 'asr': True, 'alloy-w...",GASOLINE,3800,2017,KAROQ,5,NaN,2020,AUTOMATIC,0,1,None,0,FORWARD_CONTROL,0,1897000,1104031864


In [19]:
df.id.nunique()

66234

Видно, что уникальных `id` меньше, чем всего строк в дата-сете. Сразу очистим дата-сет от дубликатов, чтобы не сохранять в файл лишние байты. И, логчино, сохраним дата-сет в файл для последующего моделирования.

In [20]:
df.drop_duplicates(subset='id', keep='first', inplace=True, ignore_index=True)

In [22]:
df.drop(columns='id').to_csv('train.csv', index=False)